# Phase 1: 기반 인프라 사용 예제

Phase 1에서 구현한 기반 인프라 컴포넌트들의 사용법을 설명합니다.

## 구현된 기능
- Configuration Models (Pydantic)
- Cycler Detector (자동 장비 타입 감지)
- Path Handler (연속 경로 검증 및 그룹화)

## 1. Configuration Models

타입 안전한 설정 객체를 제공합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.utils.config_models import ProfileConfig, CycleConfig

# Profile 분석 설정
profile_config = ProfileConfig(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=0,  # 0이면 자동 계산
    firstCrate=0.2
)

print("Profile Config:")
print(f"  Path: {profile_config.raw_file_path}")
print(f"  Min Capacity: {profile_config.mincapacity} (auto-calculate)")
print(f"  First C-rate: {profile_config.firstCrate}")

In [ ]:
# Cycle 분석 설정
cycle_config = CycleConfig(
    raw_file_path="Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc/30",
    mincapacity=0,
    firstCrate=0.2,
    chkir=False  # DCIR 계산 여부
)

print("\nCycle Config:")
print(f"  Path: {cycle_config.raw_file_path}")
print(f"  Check IR: {cycle_config.chkir}")

## 2. Cycler Detector

Rawdata 경로에서 자동으로 충방전기 타입(Toyo/PNE)을 감지합니다.

In [ ]:
from src.core.cycler_detector import detect_cycler_type, get_channel_folders

# 단일 경로에서 장비 타입 감지
toyo_path = "Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc"
cycler_type = detect_cycler_type(toyo_path)

print(f"Detected Cycler Type: {cycler_type}")
print(f"Expected: TOYO")

In [ ]:
# 채널 폴더 검색
channels = get_channel_folders(toyo_path)

print(f"\nFound {len(channels)} channels:")
for channel in channels:
    print(f"  - {channel}")

## 3. Path Handler

연속된 테스트 경로들을 검증하고 그룹화합니다.

In [ ]:
from src.utils.path_handler import PathHandler

# 연속 경로 정의
continuous_paths = [
    "Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    "Rawdata/250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc",
    "Rawdata/250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc",
    "Rawdata/250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc"
]

handler = PathHandler()

# 경로 검증
is_valid = handler.validate_continuous_paths(continuous_paths)
print(f"Paths are continuous: {is_valid}")

In [ ]:
# 채널별 경로 그룹 생성
path_groups = handler.create_path_groups(continuous_paths)

print(f"\nCreated {len(path_groups)} path groups:")
for group in path_groups:
    print(f"\nChannel: {group.channel_name}")
    print(f"  Cycler Type: {group.cycler_type}")
    print(f"  Cycle Range: {group.cycle_range_start}-{group.cycle_range_end}")
    print(f"  Paths: {len(group.paths)}")
    for path in group.paths:
        print(f"    - {path}")

## 4. 통합 예제

모든 컴포넌트를 함께 사용하는 예제입니다.

In [ ]:
# Step 1: 경로 검증
paths = [
    "Rawdata/250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    "Rawdata/250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc"
]

handler = PathHandler()
if handler.validate_continuous_paths(paths):
    print("✓ Paths validated")
    
    # Step 2: 경로 그룹 생성
    groups = handler.create_path_groups(paths)
    print(f"✓ Created {len(groups)} path groups")
    
    # Step 3: 각 그룹에 대한 설정 생성
    for group in groups:
        config = CycleConfig(
            raw_file_path=group.paths[0],  # 첫 번째 경로
            mincapacity=0,
            firstCrate=0.2,
            chkir=False
        )
        print(f"\n✓ Config created for channel {group.channel_name}")
        print(f"  Cycler: {group.cycler_type}")
        print(f"  Cycles: {group.cycle_range_start}-{group.cycle_range_end}")

## 요약

Phase 1 인프라는 다음을 제공합니다:

1. **타입 안전한 설정**: ProfileConfig, CycleConfig
2. **자동 장비 감지**: detect_cycler_type(), get_channel_folders()
3. **경로 관리**: validate_continuous_paths(), create_path_groups()

이러한 기반 위에 Phase 2에서 Profile Loader, Phase 4에서 Cycle Analyzer가 구축됩니다.